In [1]:
import sys

sys.path.append("../../")

from app.common.logger import logger
from app.config.utils import Configuration, init_config
from app.common.llm_clients.openai_client import OpenAILLMClient

from ml.data_processing.prompt import STEP_1_2_SYNTHETIC_TEXTBOOK_PROMPT

In [2]:
config = Configuration()
init_config(config)
config = config()

# 데이터 불러오기

In [3]:
import json

with open("../data/amr_guide_textbook.json", "r", encoding="utf-8") as f:
    arm_guide_textbook = json.load(f)

with open("../data/acs_alarm_textbook.json", "r", encoding="utf-8") as f:
    acs_alarm_textbook = json.load(f)

# 데이터 포맷팅 

In [21]:
system_content = "당신은 현대자동차그룹 BMA(Battery Module Assembly) 공정의 자동화 시스템을 설계한 수석 아키텍트(Chief Architect)이자 공학 교수입니다."
user_content = """BMA 공정 내 '{chapter_title}'에 대해 기술 백서 형식으로 심층 분석하여 서술해 주세요."""
think_content = """<think>\n사용자가 '{chapter_title}'에 대한 기술적 서술을 요청했다. 내장된 도메인 지식을 인출하여 상세히 설명한다.\n</think>\n\n"""

In [23]:
result = []
for j in arm_guide_textbook:
    text = j['preprocessed']
    chapter_title = text.split('\n')[0].strip().replace('#', '').strip()
    conversation = {
        "conversation": [
            {
                "role": "system",
                "content": system_content
            },
            {
                "role": "user",
                "content": user_content.format(chapter_title=chapter_title)
            },
            {
                "role": "assistant",
                "content": think_content.format(chapter_title=chapter_title) + text
            }
        ]
    }
    result.append(conversation)


In [24]:
with open("arm_guide_qwen_conversation_format.json", "w", encoding="utf-8") as f:
    json.dump(result, f, ensure_ascii=False, indent=4)

In [25]:
result2 = []
for j in acs_alarm_textbook:
    text = j['preprocessed']
    chapter_title = text.split('\n')[0].strip().replace('#', '').strip()
    conversation = {
        "conversation": [
            {
                "role": "system",
                "content": system_content
            },
            {
                "role": "user",
                "content": user_content.format(chapter_title=chapter_title)
            },
            {
                "role": "assistant",
                "content": think_content.format(chapter_title=chapter_title) + text
            }
        ]
    }
    result2.append(conversation)

with open("acs_alarm_qwen_conversation_format.json", "w", encoding="utf-8") as f:
    json.dump(result2, f, ensure_ascii=False, indent=4)